In [1]:
pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import sys

In [3]:
sys.path.append('../cfg')
from cfg.database_cfg import cfg

In [4]:
EDITING=False

In [31]:
con = create_engine('mysql://%s@%s/%s?charset=%s&use_unicode=%s&passwd=%s' % (cfg.user, cfg.host, cfg.dbname+'_prepped', cfg.charset, cfg.use_unicode, cfg.passwd), pool_recycle=3600)
climbdf = pd.read_sql("SELECT * from climb_prepped", con, index_col='index')
areadf = pd.read_sql("SELECT * from area_prepped", con, index_col='index')
climberdf = pd.read_sql("SELECT * from climber_prepped", con, index_col='index')
tickdf = pd.read_sql("SELECT * from ticks_prepped", con, index_col='index')
commentdf = pd.read_sql("SELECT * from comments_prepped", con, index_col='index')
gradedf = pd.read_sql("SELECT * from grades_prepped", con, index_col='index')
stardf = pd.read_sql("SELECT * from stars_prepped", con, index_col='index')
hitsdf = pd.read_sql("SELECT * from hits_prepped", con, index_col='index')


In [6]:
def cluster(df, colname, clustertype='kmeans', k=10):
    from sklearn.cluster import KMeans, AffinityPropagation
    matrix=df[[col for col in df.columns if col != colname]].values
    labels=df[colname].values
    if clustertype=='affinity':
        clf=AffinityPropagation()
    elif clustertype=="kmeans":
        clf=KMeans(n_clusters=k)
    clusters=clf.fit_predict(matrix)
    return pd.DataFrame(data={colname:labels,'cluster':clusters})

In [8]:
import json
import collections
with open('cfg/apriori.json') as f:
    apriori=json.loads(f.read())
with open('cfg/attributes.json') as f:
    attributes=json.loads(f.read(), object_pairs_hook=collections.OrderedDict)
    

In [42]:
climberdf.loc[climberdf['name']=='climber','mainarea']=880

In [337]:
if EDITING:
    with open('cfg/apriori.json', 'wb') as f:
        json.dump(apriori, f)
    with open('cfg/attributes.json', 'wb') as f:
        json.dump(attributes, f)

In [35]:
if EDITING:
    chunksize=1000
    areadf.to_sql('area_prepped', con, if_exists='replace', chunksize=chunksize)
    climberdf.to_sql('climber_prepped', con,if_exists='replace', chunksize=chunksize)
    tickdf.to_sql('ticks_prepped', con, if_exists='replace', chunksize=chunksize)
    commentdf.to_sql('comments_prepped', con, if_exists='replace', chunksize=chunksize)
    gradedf.to_sql('grades_prepped', con, if_exists='replace', chunksize=chunksize)
    stardf.to_sql('stars_prepped', con, if_exists='replace', chunksize=chunksize)
    hitsdf.to_sql('hits_prepped', con, if_exists='replace', chunksize=chunksize)
    climbdf.to_sql('climb_prepped', con, if_exists='replace', chunksize=chunksize)

In [43]:
climberdf.to_sql('climber_prepped', con,if_exists='replace', chunksize=chunksize)